## Assignment 3: Name Count
Make WordCount program for all the names in the dataset. Name is a word with the following properties:

- The first character is not a digit (other characters can be digits).
- The first character is uppercase, all the other characters that are letters are lowercase.
- There are less than 0.5% occurrences of this word, when this word regardless to its case appears in the dataset and the condition (2) is not met.
Order by quantity, most popular first, output format:
```
name <tab> count
```
The result is the 5th line in the output, for example:
```
john 1234
```

In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

def is_name(word):
    if len(word) < 2:
        return False
    elif (word[0].isalpha()) and (word[0].isupper()) and (word[1:].islower()):
        return True
    else:
        return False

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        name_flag = int(is_name(word))
        print "%s\t%d\t%d" % (word.lower(), 1, name_flag)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

def condition_name(cnt, cnt_name):
    if (cnt - cnt_name) / float(cnt) * 100 < 0.5:
        return True
    else:
        return False

current_key = None
current_cnt = 0
current_cnt_name = 0

for line in sys.stdin:
    try:
        key, cnt, cnt_name = unicode(line.strip()).split('\t')
        cnt = int(cnt)
        cnt_name = int(cnt_name)
    except ValueError as e:
        continue
    
    if current_key != key:
        if current_key and condition_name(current_cnt, current_cnt_name):
            print "%s\t%d" % (current_key, current_cnt)
        current_key = key
        current_cnt = cnt
        current_cnt_name = cnt_name
    else:
        current_cnt += cnt
        current_cnt_name += cnt_name
        
print "%s\t%d" % (current_key, current_cnt)

Overwriting reducer.py


In [3]:
%%writefile mapper_2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%d\t%s" % (count, word)

Writing mapper_2.py


In [4]:
%%writefile reducer_2.py

import sys

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
        
    print "%s\t%d" % (word, count)

Writing reducer_2.py


In [5]:
%%bash

OUT_DIR="name_count"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming name count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
OUT_DIR_2="name_count_sorted"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming name count sorting" \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D map.output.key.field.separator=\t \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_2.py,reducer_2.py \
    -mapper "python mapper_2.py" \
    -reducer "python reducer_2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR_2} > /dev/null
    
hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head -5 | tail -1

american	17217
english	7797
british	6670
john	5766
french	5753
german	4622
york	4456
roman	3824
europe	3587
european	3528


18/04/13 08:20:06 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/13 08:20:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/13 08:20:07 INFO mapred.FileInputFormat: Total input files to process : 1
18/04/13 08:20:07 INFO mapreduce.JobSubmitter: number of splits:2
18/04/13 08:20:07 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1523567791799_0026
18/04/13 08:20:07 INFO impl.YarnClientImpl: Submitted application application_1523567791799_0026
18/04/13 08:20:07 INFO mapreduce.Job: The url to track the job: http://8c786766d7b1:8088/proxy/application_1523567791799_0026/
18/04/13 08:20:07 INFO mapreduce.Job: Running job: job_1523567791799_0026
18/04/13 08:20:13 INFO mapreduce.Job: Job job_1523567791799_0026 running in uber mode : false
18/04/13 08:20:13 INFO mapreduce.Job:  map 0% reduce 0%
18/04/13 08:20:29 INFO mapreduce.Job:  map 43% reduce 0%
18/04/13 08:20:35 INFO mapreduce.Job:  map 60% reduce 0%
18/04/13 08:20:41 INFO 